In [1]:
    import cv2

    cap = cv2.VideoCapture(r"C:\Users\Lam Long\Downloads\Khoa học nhận thức\AI-Powered-Entrance-Monitoring-People-Counting\test3.mp4")
    r,frame = cap.read()

    points = []


    def draw_line(event,x,y,flags,param):
        if event == cv2.EVENT_LBUTTONDOWN:
            points.append((x,y))
            print(f"Points Selected is {(x,y)}")
            if len(points) ==2:
                print(f"Line Drawn from {points[0]} and {points[1]}")
            
    cv2.namedWindow("Select the Points")
    cv2.setMouseCallback("Select the Points",draw_line)
            
            
    while True:
        if not r:
            print("Unable to load the frame.")
            break
        temp_frame = frame.copy()
        if len(points) == 1:
            cv2.circle(temp_frame,points[0],5,(0,255,0),-1)
        elif len(points) == 2:
            cv2.line(temp_frame,points[0],points[1],(255,255,0),1)
        cv2.imshow("Select the Points",temp_frame)
        if cv2.waitKey(1) & 0xff == ord("q"):
            break
    cap.release()
    cv2.destroyAllWindows()

Points Selected is (232, 159)
Points Selected is (257, 203)
Line Drawn from (232, 159) and (257, 203)


In [4]:
### HEATMAP 1 (OKE HON)
import cv2
import numpy as np
from ultralytics import YOLO
import datetime

# Load model + video
model = YOLO("yolov8n.pt")
cap = cv2.VideoCapture(r"C:\Users\Lam Long\Downloads\Khoa học nhận thức\AI-Powered-Entrance-Monitoring-People-Counting\test2.mp4")

fps = cap.get(cv2.CAP_PROP_FPS)
w = int(cap.get(3))
h = int(cap.get(4))

# Lưu video ghép 2 khung
out = cv2.VideoWriter("output_heatmap_final2.mp4", cv2.VideoWriter_fourcc(*"mp4v"), fps, (w*2, h))

# Cấu hình đếm người
line_x = 232
line_y1, line_y2 = 159, 203
max_people_bar = 20

track_history = {}
in_count = out_count = 0
person_index = {}


# ========================= HEATMAP TỪ BOUNDING BOX =========================
def generate_heatmap_from_bboxes(shape, boxes, scale=1.5):
    heatmap = np.zeros((shape[0], shape[1]), dtype=np.float32)
    
    for (x1, y1, x2, y2) in boxes:
        cx = (x1 + x2) // 2
        cy = (y1 + y2) // 2

        # Vùng ảnh hưởng to theo kích thước người
        w = (x2 - x1) * scale
        h = (y2 - y1) * scale

        x_grid, y_grid = np.meshgrid(
            np.arange(shape[1]),
            np.arange(shape[0])
        )

        gaussian = np.exp(
            -(((x_grid - cx) ** 2) / (2 * (w/2) ** 2)
            + ((y_grid - cy) ** 2) / (2 * (h/2) ** 2))
        )

        heatmap += gaussian

    heatmap = np.clip(heatmap * 255, 0, 255).astype(np.uint8)
    return heatmap


# Log mỗi ~1 giây
log_counter = 0


# ============================== MAIN LOOP ==============================
while True:
    ret, frame = cap.read()
    if not ret:
        break

    frame_original = frame.copy()
    centers = []
    bboxes = []  # NEW → lưu bbox để tạo heatmap crowd

    results = model.track(frame, persist=True, conf=0.5, classes=[0], tracker="botsort.yaml")

    for r in results:
        for box in r.boxes:
            if box.id is None:
                continue

            x1, y1, x2, y2 = map(int, box.xyxy[0])
            cx = (x1 + x2) // 2
            cy = (y1 + y2) // 2
            track_id = int(box.id)

            bboxes.append((x1, y1, x2, y2))   # → Thêm vào heatmap

            if track_id not in person_index:
                person_index[track_id] = len(person_index) + 1

            label = f"PERSON {person_index[track_id]} (ID:{track_id})"

            # Vẽ bbox
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
            (tw, th), _ = cv2.getTextSize(label, cv2.FONT_HERSHEY_SIMPLEX, 0.6, 2)
            cv2.rectangle(frame, (x1, y1 - th - 8), (x1 + tw + 4, y1), (0, 0, 0), -1)
            cv2.putText(frame, label, (x1, y1 - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)

            centers.append((cx, cy))

            # Đếm vào/ra
            if track_id not in track_history:
                track_history[track_id] = cx
                continue

            prev_cx = track_history[track_id]
            track_history[track_id] = cx

            if prev_cx < line_x <= cx:
                in_count += 1
            elif prev_cx > line_x >= cx:
                out_count += 1

    current_inside = in_count - out_count


    # ===================== HUD BÊN TRÁI =====================
    overlay = frame.copy()
    cv2.rectangle(overlay, (10, 10), (w - 10, 170), (0, 0, 0), -1)
    frame = cv2.addWeighted(overlay, 0.55, frame, 0.45, 0)

    cv2.putText(frame, "HE THONG AI DEM NGUOI RA VAO", (25, 40),
                cv2.FONT_HERSHEY_DUPLEX, 0.9, (255, 255, 255), 2)
    cv2.putText(frame, datetime.datetime.now().strftime("%d/%m/%Y %H:%M:%S"),
                (w - 300, 40), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (220, 220, 220), 2)

    cv2.line(frame, (line_x, line_y1), (line_x, line_y2), (255, 0, 255), 3)

    cv3 = cv2
    cv2.putText(frame, f"IN: {in_count}", (40, 100),
                cv2.FONT_HERSHEY_SIMPLEX, 1.4, (0, 255, 0), 4)
    cv2.putText(frame, f"OUT: {out_count}", (40, 155),
                cv2.FONT_HERSHEY_SIMPLEX, 1.4, (0, 0, 255), 4)
    cv2.putText(frame, f"HIEN TAI: {current_inside}", (300, 130),
                cv2.FONT_HERSHEY_SIMPLEX, 1.2, (255, 255, 0), 3)

    bar_length = max(0, min(int(current_inside / max_people_bar * 400), 400))
    cv2.rectangle(frame, (300, 150), (700, 170), (80, 80, 80), 2)
    cv2.rectangle(frame, (300, 150), (300 + bar_length, 170), (0, 255, 255), -1)


    # ===================== HEATMAP CROWDED BÊN PHẢI =====================
    heatmap_gray = generate_heatmap_from_bboxes(frame_original.shape[:2], bboxes, scale=1.2)

    # Màu heatmap đẹp nhất: TURBO
    heatmap_color = cv2.applyColorMap(heatmap_gray, cv2.COLORMAP_TURBO)

    heatmap_overlay = cv2.addWeighted(frame_original, 0.45, heatmap_color, 0.65, 0)

    estimated = len(bboxes)  # ổn định – không nhảy

    # HUD bên heatmap
    cv2.putText(heatmap_overlay, "DENSITY HEATMAP", (20, 50),
                cv2.FONT_HERSHEY_DUPLEX, 1.2, (255, 255, 255), 3)
    cv2.putText(heatmap_overlay, f"Est: {estimated}", (20, 110),
                cv2.FONT_HERSHEY_DUPLEX, 2.0, (0, 255, 255), 5)

    combined = np.hstack((frame, heatmap_overlay))


    # ===================== LOG & HIỂN THỊ =====================
    log_counter += 1
    if log_counter % 30 == 0:
        ratio = current_inside / 18
        status = "VANG" if ratio <= 0.3 else "BINH THUONG" if ratio <= 0.7 else "DONG"
        print(f"[INFO] {datetime.datetime.now().strftime('%H:%M:%S')} | Trong: {current_inside} nguoi → {status}")

    cv2.imshow("HE THONG AI DEM NGUOI + HEATMAP", combined)
    out.write(combined)

    if cv2.waitKey(1) == ord('q'):
        break

cap.release()
out.release()
cv2.destroyAllWindows()
print("HOÀN TẤT! File đã lưu: output_heatmap_final2.mp4")



0: 384x640 16 persons, 77.3ms
Speed: 55.3ms preprocess, 77.3ms inference, 16.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 13 persons, 48.5ms
Speed: 2.3ms preprocess, 48.5ms inference, 2.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 12 persons, 44.3ms
Speed: 2.5ms preprocess, 44.3ms inference, 2.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 11 persons, 43.3ms
Speed: 2.1ms preprocess, 43.3ms inference, 2.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 11 persons, 44.1ms
Speed: 2.9ms preprocess, 44.1ms inference, 1.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 8 persons, 38.4ms
Speed: 2.5ms preprocess, 38.4ms inference, 2.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 10 persons, 58.2ms
Speed: 2.1ms preprocess, 58.2ms inference, 2.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 14 persons, 39.4ms
Speed: 2.3ms preprocess, 39.4ms inference, 3.1ms postprocess per image a

In [1]:
### CODE CHUAN
import cv2
from ultralytics import YOLO
import datetime

# Load model YOLO
model = YOLO("yolov8n.pt")

# Load video
cap = cv2.VideoCapture(r"C:\Users\Lam Long\Downloads\Khoa học nhận thức\AI-Powered-Entrance-Monitoring-People-Counting\test.mp4")

fps = cap.get(cv2.CAP_PROP_FPS)
fourcc = cv2.VideoWriter_fourcc(*"mp4v")
out = cv2.VideoWriter("outputfinal.mp4", fourcc, fps, (int(cap.get(3)), int(cap.get(4))))

# ---------------------------
# CẤU HÌNH
# ---------------------------
# Đường đếm
line_x = 399          # tọa độ x đường đếm
line_y1, line_y2 = 252, 560

# Biến lưu tracking
track_history = {}
in_count = 0
out_count = 0

# Gán tên PERSON số thứ tự
person_index = {}

# ---------------------------
# MAIN LOOP
# ---------------------------
while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Dùng BoT-SORT qua YOLO
    results = model.track(
        frame,
        persist=True,
        conf=0.5,
        classes=[0],
        tracker="botsort.yaml"
    )

    if results is None:
        continue

    for r in results:
        for box in r.boxes:
            if box.id is None:
                continue

            # BBOX
            x1, y1b, x2, y2b = map(int, box.xyxy[0])
            cx = (x1 + x2) // 2
            cy = (y1b + y2b) // 2
            track_id = int(box.id)

            # GÁN PERSON INDEX CHO TRACK_ID
            if track_id not in person_index:
                person_index[track_id] = len(person_index) + 1

            label_text = f"PERSON {person_index[track_id]} (ID:{track_id})"

            # VẼ BOX
            cv2.rectangle(frame, (x1, y1b), (x2, y2b), (0, 255, 0), 2)

            # Vẽ nền label
            (tw, th), _ = cv2.getTextSize(label_text, cv2.FONT_HERSHEY_SIMPLEX, 0.6, 2)
            cv2.rectangle(frame, (x1, y1b - th - 8), (x1 + tw + 4, y1b), (0, 0, 0), -1)
            cv2.putText(frame, label_text, (x1, y1b - 5),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)

            # ------------- ĐẾM NGƯỜI VÀO/RA -------------
            if track_id not in track_history:
                track_history[track_id] = cx
                continue

            prev_cx = track_history[track_id]
            track_history[track_id] = cx

            moved_left_to_right = prev_cx < line_x and cx >= line_x
            moved_right_to_left = prev_cx > line_x and cx <= line_x

            if moved_left_to_right:
                in_count += 1

            elif moved_right_to_left:
                out_count += 1

    # ------------------- HUD ---------------------
    overlay = frame.copy()
    cv2.rectangle(overlay, (10, 10), (frame.shape[1] - 10, 170), (0, 0, 0), -1)
    frame = cv2.addWeighted(overlay, 0.55, frame, 0.45, 0)

    # TITLE
    cv2.putText(frame, "HE THONG AI DEM NGUOI RA VAO",
                (25, 40), cv2.FONT_HERSHEY_DUPLEX, 0.9, (255, 255, 255), 2)

    # TIME
    cv2.putText(frame, datetime.datetime.now().strftime("%d/%m/%Y %H:%M:%S"),
                (frame.shape[1] - 300, 40),
                cv2.FONT_HERSHEY_SIMPLEX, 0.7, (220, 220, 220), 2)

    # Vẽ line
    cv2.line(frame, (line_x, line_y1), (line_x, line_y2), (255, 0, 255), 3)

    # HIỆN THỊ IN / OUT
    cv2.putText(frame, f"IN: {in_count}", (40, 100),
                cv2.FONT_HERSHEY_SIMPLEX, 1.4, (0, 255, 0), 4)

    cv2.putText(frame, f"OUT: {out_count}", (40, 155),
                cv2.FONT_HERSHEY_SIMPLEX, 1.4, (0, 0, 255), 4)

    # HIEN TAI
    current_inside = in_count - out_count
    cv2.putText(frame, f"HIEN TAI: {current_inside}", (300, 130),
                cv2.FONT_HERSHEY_SIMPLEX, 1.2, (255, 255, 0), 3)

    # BAR BIỂU ĐỒ
    bar_start, bar_end = 300, 700
    max_people = 20
    bar_length = int((current_inside / max_people) * (bar_end - bar_start))
    bar_length = max(0, min(bar_length, bar_end - bar_start))

    cv2.rectangle(frame, (bar_start, 150), (bar_end, 170), (80, 80, 80), 2)
    cv2.rectangle(frame, (bar_start, 150), (bar_start + bar_length, 170), (0, 255, 255), -1)

    # SHOW VIDEO
    cv2.imshow("People IN_OUT Counter", frame)
    out.write(frame)

    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

out.release()
cap.release()
cv2.destroyAllWindows()

print("Xong! Da luu file outputfinal.mp4")



0: 384x640 (no detections), 103.8ms
Speed: 8.6ms preprocess, 103.8ms inference, 11.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 61.3ms
Speed: 3.2ms preprocess, 61.3ms inference, 0.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 53.5ms
Speed: 2.2ms preprocess, 53.5ms inference, 0.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 55.3ms
Speed: 2.3ms preprocess, 55.3ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 61.9ms
Speed: 2.1ms preprocess, 61.9ms inference, 0.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 52.3ms
Speed: 2.5ms preprocess, 52.3ms inference, 0.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 51.7ms
Speed: 2.8ms preprocess, 51.7ms inference, 0.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 57.6ms
Speed: 2.1ms preprocess, 57.6ms infer

In [18]:
import cv2
import numpy as np
from ultralytics import YOLO
import datetime
import sys # Thư viện để thoát chương trình sạch sẽ

# ================= CẤU HÌNH =================
# Nếu muốn nhanh hơn, dùng 'yolov8n.pt'. Muốn chính xác hơn, dùng 'yolov8s.pt'
model = YOLO("yolov8s.pt") 
video_path = r"C:\Users\Lam Long\Downloads\Khoa học nhận thức\AI-Powered-Entrance-Monitoring-People-Counting\test2.mp4"

cap = cv2.VideoCapture(video_path)

# Lấy thông số video để tính toán thời điểm dừng
fps = cap.get(cv2.CAP_PROP_FPS)
w = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
h = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT)) # Tổng số khung hình của video

print(f"--- BẮT ĐẦU XỬ LÝ ---")
print(f"Tổng số frame: {total_frames}")
print(f"Kích thước: {w}x{h} | FPS gốc: {fps}")

out = cv2.VideoWriter("output_heatmap_auto_stop.mp4", cv2.VideoWriter_fourcc(*"mp4v"), fps, (w, h))

# ========================= HEATMAP FUNCTION =========================
def generate_heatmap_from_bboxes(shape, boxes, scale=1.5):
    heatmap = np.zeros((shape[0], shape[1]), dtype=np.float32)
    for (x1, y1, x2, y2) in boxes:
        cx, cy = (x1 + x2) // 2, (y1 + y2) // 2
        w_box, h_box = (x2 - x1) * scale, (y2 - y1) * scale
        if w_box <= 0 or h_box <= 0: continue
        
        # Tạo Gaussian nhanh hơn bằng cách chỉ tính trong vùng ROI (tối ưu tốc độ)
        # (Ở đây giữ nguyên logic cũ cho đơn giản, nhưng nếu lag có thể tối ưu đoạn này)
        x_grid, y_grid = np.meshgrid(np.arange(shape[1]), np.arange(shape[0]))
        gaussian = np.exp(-(((x_grid - cx) ** 2) / (2 * (w_box/2) ** 2) + ((y_grid - cy) ** 2) / (2 * (h_box/2) ** 2)))
        heatmap += gaussian
    return np.clip(heatmap * 255, 0, 255).astype(np.uint8)

frame_count = 0

# ============================== MAIN LOOP ==============================
while True:
    ret, frame = cap.read()
    
    # 1. ĐIỀU KIỆN DỪNG: Nếu không đọc được frame (hết video) -> Thoát
    if not ret:
        print("\n[THÔNG BÁO] Đã chạy hết video gốc. Đang lưu file...")
        break
    
    # 2. ĐIỀU KIỆN DỪNG PHỤ: Nếu số frame chạy vượt quá tổng frame (tránh lỗi lặp vô hạn)
    frame_count += 1
    if total_frames > 0 and frame_count > total_frames:
        print("\n[THÔNG BÁO] Đã đạt giới hạn frame. Đang lưu file...")
        break

    frame_original = frame.copy()
    bboxes = []

    # Tracking
    results = model.track(frame, persist=True, conf=0.25, iou=0.6, classes=[0], tracker="bytetrack.yaml", verbose=False)

    for r in results:
        for box in r.boxes:
            if box.id is None: continue
            x1, y1, x2, y2 = map(int, box.xyxy[0])
            if (x2 - x1) > 10 and (y2 - y1) > 20:
                bboxes.append((x1, y1, x2, y2))
                # Không vẽ khung xanh nữa cho đỡ rối, chỉ tập trung heatmap

    # Heatmap & Overlay
    heatmap_gray = generate_heatmap_from_bboxes(frame_original.shape[:2], bboxes, scale=1.5)
    heatmap_color = cv2.applyColorMap(heatmap_gray, cv2.COLORMAP_TURBO)
    combined_frame = cv2.addWeighted(frame_original, 0.6, heatmap_color, 0.4, 0)

    # HUD & Stats
    current_people_count = len(bboxes)
    status = "VANG" if current_people_count <= 5 else "DONG (WARNING)" if current_people_count > 30 else "BINH THUONG"
    color_status = (0, 255, 0) if current_people_count <= 5 else (0, 0, 255) if current_people_count > 15 else (0, 255, 255)

# ===================== GIAO DIỆN TINH GỌN (THEO YÊU CẦU MỚI) =====================
    img_h, img_w = combined_frame.shape[:2]

    # --- PHẦN 1: KHUNG TRÁI (ÁP SÁT GÓC & COMPACT) ---
    # Panel dính sát góc trái trên (0,0)
    panel_x, panel_y = 0, 0
    # Tăng chiều rộng lên 240 để đủ chỗ cho chữ "EST PEOPLE"
    panel_w, panel_h = 240, 110 
    
    sub_img = combined_frame.copy()
    
    # Nền panel trái (Mờ tối)
    cv2.rectangle(sub_img, (panel_x, panel_y), (panel_x + panel_w, panel_y + panel_h), (0, 0, 0), -1)
    
    # Thanh trạng thái (Accent Bar)
    cv2.rectangle(sub_img, (panel_x, panel_y), (panel_x + 5, panel_y + panel_h), color_status, -1)
    
    # Hòa trộn nền (Transparency)
    combined_frame = cv2.addWeighted(sub_img, 0.6, combined_frame, 0.4, 0)

    # Hàm vẽ chữ có viền (Outline)
    def draw_text_outline(img, text, pos, font, scale, color, thickness):
        x, y = pos
        cv2.putText(img, text, (x, y), font, scale, (0, 0, 0), thickness + 2) # Viền đen
        cv2.putText(img, text, (x, y), font, scale, color, thickness)           # Chữ chính

    # --- NỘI DUNG ---
    
    # 1. TIÊU ĐỀ
    draw_text_outline(combined_frame, "CROWD DENSITY AI", (panel_x + 15, panel_y + 30), 
                      cv2.FONT_HERSHEY_DUPLEX, 0.6, (255, 255, 255), 1)
    
    # 2. EST PEOPLE: [Số]
    # Vẽ chữ "EST PEOPLE:" màu xám nhạt
    draw_text_outline(combined_frame, "EST PEOPLE:", (panel_x + 15, panel_y + 65), 
                      cv2.FONT_HERSHEY_SIMPLEX, 0.55, (180, 180, 180), 1)
    # Vẽ số lượng (To và sáng hơn) ngay bên cạnh
    draw_text_outline(combined_frame, f"{current_people_count}", (panel_x + 140, panel_y + 65), 
                      cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)

    # 3. STATUS: [Trạng thái]
    # Vẽ chữ "STATUS:" màu xám nhạt
    draw_text_outline(combined_frame, "STATUS:", (panel_x + 15, panel_y + 90), 
                      cv2.FONT_HERSHEY_SIMPLEX, 0.55, (180, 180, 180), 1)
    # Vẽ trạng thái (Màu theo cảnh báo)
    draw_text_outline(combined_frame, f"{status}", (panel_x + 95, panel_y + 90), 
                      cv2.FONT_HERSHEY_SIMPLEX, 0.55, color_status, 2)


    # --- PHẦN 2: NGÀY GIỜ (GÓC PHẢI - KHÔNG NỀN) ---
    time_str = datetime.datetime.now().strftime("%H:%M:%S  %d/%m/%Y")
    
    (t_w, t_h), _ = cv2.getTextSize(time_str, cv2.FONT_HERSHEY_SIMPLEX, 0.65, 2)
    time_x = img_w - t_w - 20 
    time_y = 40               

    draw_text_outline(combined_frame, time_str, (time_x, time_y), 
                      cv2.FONT_HERSHEY_SIMPLEX, 0.65, (255, 255, 255), 2)
    # Hiển thị thanh tiến độ ngay trên Video để bạn thấy nó đang chạy
    if total_frames > 0:
        progress = frame_count / total_frames
        bar_w = int(progress * w)
        cv2.rectangle(combined_frame, (0, h-10), (w, h), (50, 50, 50), -1) # Nền thanh loading
        cv2.rectangle(combined_frame, (0, h-10), (bar_w, h), (0, 255, 0), -1) # Thanh màu xanh

    # Log tiến độ ra Terminal (để bạn yên tâm là nó không bị treo)
    sys.stdout.write(f"\r[Đang xử lý] Frame: {frame_count}/{total_frames} ({frame_count/total_frames*100:.1f}%)")
    sys.stdout.flush()

    cv2.imshow("AI HEATMAP (Bam Q de thoat som)", combined_frame)
    out.write(combined_frame)

    if cv2.waitKey(1) == ord('q'):
        print("\n[THÔNG BÁO] Người dùng bấm Q để dừng sớm.")
        break

# Dọn dẹp
cap.release()
out.release()
cv2.destroyAllWindows()
print(f"\nHOÀN TẤT! File video đã được lưu tại: output_heatmap_auto_stop.mp4")

--- BẮT ĐẦU XỬ LÝ ---
Tổng số frame: 424
Kích thước: 768x432 | FPS gốc: 29.97002997002997
[Đang xử lý] Frame: 424/424 (100.0%)
[THÔNG BÁO] Đã chạy hết video gốc. Đang lưu file...

HOÀN TẤT! File video đã được lưu tại: output_heatmap_auto_stop.mp4
